In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
from topgun.src.risk.factor_returns import bloomberg_factor_returns
from topgun.src.risk.definition import riskmodel

riskmodel

ModuleNotFoundError: No module named 'bbq'

macro0
macro1
macro2
equity1
equity2


In [31]:

facts = pd.DataFrame()

riskmodels_ = list(riskmodel.keys())
for riskmodel_ in riskmodels_:
    print(riskmodel_)
    factor_returns = bloomberg_factor_returns(model=riskmodel[riskmodel_]).fillna(0)
    factor_returns = factor_returns.melt(var_name="factor_name", value_name="return_value", ignore_index=False).reset_index(names="date").assign(riskmodel_code=riskmodel_)
    facts = pd.concat([facts, factor_returns], ignore_index=True)
facts


macro0
macro1
macro2
equity1
equity2


,date,factor_name,return_value,riskmodel_code
0,2019-05-31,Market,0.00000,macro0
1,2019-06-03,Market,-0.00058,macro0
2,2019-06-04,Market,0.01594,macro0
3,2019-06-05,Market,0.00875,macro0
4,2019-06-06,Market,0.00398,macro0
...,...,...,...,...
34246,2025-08-25,RE,-0.01315,equity2
34247,2025-08-26,RE,-0.00324,equity2
34248,2025-08-27,RE,0.00908,equity2
34249,2025-08-28,RE,-0.02163,equity2


In [ ]:

facts.to_parquet('data/factor_returns_all.parquet')


In [ ]:
df = pd.read_parquet('data/data.pqt')
df

,date,node,measure,value
0,2020-07-01,AS,Contribution,0.000000e+00
1,2020-07-02,AS,Contribution,0.000000e+00
2,2020-07-03,AS,Contribution,0.000000e+00
3,2020-07-06,AS,Contribution,0.000000e+00
4,2020-07-07,AS,Contribution,0.000000e+00
...,...,...,...,...
281659,2025-06-24,TOTAL,Active Weight Vs ÖT,4.832135e-12
281660,2025-06-25,TOTAL,Active Weight Vs ÖT,4.465761e-12
281661,2025-06-26,TOTAL,Active Weight Vs ÖT,3.098855e-12
281662,2025-06-27,TOTAL,Active Weight Vs ÖT,2.189138e-12


In [18]:
import pandas as pd

def build_hierarchy_paths_with_leaf_flag(
    df: pd.DataFrame, child_col="CHILD", parent_col="PARENT", sep="/"
) -> pd.DataFrame:
    """
    Adds two columns:
      - PATH: full path from root
      - IS_LEAF: True if the node has no children, False otherwise
    """
    # normalize
    ch = df[child_col].astype(str).str.strip()
    pa = df[parent_col].astype(str).str.strip().where(df[parent_col].notna(), "")
    parent_of = dict(zip(ch, pa))

    cache = {}

    def path_to(node: str, seen=None) -> str:
        if node in cache:
            return cache[node]
        if seen is None:
            seen = set()
        if node in seen:
            raise ValueError(f"Cycle detected at '{node}'")
        seen.add(node)

        parent = parent_of.get(node, "")
        if (not parent) or (parent not in parent_of):  # root or dangling ancestor
            cache[node] = node
            return cache[node]

        full = path_to(parent, seen) + sep + node
        cache[node] = full
        return full

    # compute paths
    paths = {node: path_to(node) for node in parent_of.keys()}

    # leaf = child that is never a parent
    non_leafs = set(parent_of.values()) - {""}
    is_leaf = {node: node not in non_leafs for node in parent_of.keys()}

    # merge back
    out = df.copy()
    out["PATH"] = out[child_col].map(paths)
    out["IS_LEAF"] = out[child_col].map(is_leaf)
    return out


# --- Example usage ---
child_parent = [
    ("TOTAL",   ""),
    ("EQLIKE",  "TOTAL"),
    ("EQ",      "EQLIKE"),
    ("EQSE",    "EQ"),
    ("EQSELC",  "EQSE"),
    ("EQSESC",  "EQSE"),
    ("EQDM",    "EQ"),
    ("EQDMLC",  "EQDM"),
    ("EQDMSC",  "EQDM"),
    ("EQEM",    "EQ"),
    ("HY",      "EQLIKE"),
    ("RE",      "EQLIKE"),
    ("RELI",    "RE"),
    ("RENL",    "RE"),
    ("IN",      "EQLIKE"),
    ("INLI",    "IN"),
    ("INNL",    "IN"),
    ("PE",      "EQLIKE"),
    ("HF",      "EQLIKE"),
    ("AS",      "EQLIKE"),
    ("OVL",     "EQLIKE"),
    ("IG",      "TOTAL"),
    ("CA",      "TOTAL"),
    ("CCY",     "TOTAL"),
]
df = pd.DataFrame(child_parent, columns=["CHILD", "PARENT"])

path_table = build_hierarchy_paths_with_leaf_flag(df)
path_table

,CHILD,PARENT,PATH,IS_LEAF
0,TOTAL,,TOTAL,False
1,EQLIKE,TOTAL,TOTAL/EQLIKE,False
2,EQ,EQLIKE,TOTAL/EQLIKE/EQ,False
3,EQSE,EQ,TOTAL/EQLIKE/EQ/EQSE,False
4,EQSELC,EQSE,TOTAL/EQLIKE/EQ/EQSE/EQSELC,True
5,EQSESC,EQSE,TOTAL/EQLIKE/EQ/EQSE/EQSESC,True
6,EQDM,EQ,TOTAL/EQLIKE/EQ/EQDM,False
7,EQDMLC,EQDM,TOTAL/EQLIKE/EQ/EQDM/EQDMLC,True
8,EQDMSC,EQDM,TOTAL/EQLIKE/EQ/EQDM/EQDMSC,True
9,EQEM,EQ,TOTAL/EQLIKE/EQ/EQEM,True


In [21]:


def get_latest_metric(df, component_id, metric_name):
    """
    Fetch the latest value for a given metric for a specific component.
    """
    return df.loc[component_id, metric_name].iloc[-1]

def get_historical_metric(df, component_id, metric_name):
    """
    Fetch historical returns for a given component and metric.
    """
    return pd.Series(data=df.loc[component_id, metric_name], name=component_id)

def pre_process_pearl_data(path: str, relative_to: str, overlays: list[str]) -> pd.DataFrame:
    """
    Pre-process the data to create a pivoted DataFrame with portfolio and benchmark returns.
    """
    assert relative_to in ['SP', 'ST', 'ÖT']
    df = pd.read_parquet(path)
    df['date'] = pd.to_datetime(df['date'])
    df = df.pivot(index=['node', 'date'], columns='measure', values='value')
    df['portfolio_return'] = df['Return']
    df['benchmark_return'] = df['Return'] - df[f'Active Return Vs {relative_to}']
    df['portfolio_weight'] = df['Weight']
    df['benchmark_weight'] = df['Weight'] - df[f'Active Weight Vs {relative_to}']
    df['contribution'] = df['Contribution']

    if overlays is not None:
        for overlay in overlays:
            print(f"Processing overlay {overlay}")
            # Use contribution as portfolio return instead
            df.loc[overlay, 'portfolio_return'] = df.loc[overlay, 'Contribution'].to_numpy()
            df.loc[overlay, 'benchmark_return'] = df.loc[overlay, 'Contribution'].to_numpy() * 0.00001
            df.loc[overlay, 'portfolio_weight'] = 0.0
            df.loc[overlay, 'benchmark_weight'] = 0.0
        

    columns = [
        'portfolio_return', 'benchmark_return', 'portfolio_weight', 'benchmark_weight', 'Contribution']

    return df[columns]

overlays = ['CCY', 'OVL']



for vs_to in ['SP', 'ST', 'ÖT']:
    df = pre_process_pearl_data(path='data/data.pqt', relative_to=vs_to, overlays=overlays).reset_index()
    # drop node (index) that contain ['INNL','RENL','AS']
    df = df[~df['node'].isin(['INNL','RENL','AS'])]# ,'EQEM', 'RELI','INLI', 'PE', 'RE', 'IN'])]
    df = df.merge(path_table, left_on='node', right_on='CHILD', how='left')
    
    df = df.rename(columns={"PATH": "component_id"})
    df = df[['date', 'component_id', 'portfolio_return', 'benchmark_return', 'portfolio_weight', 'benchmark_weight']]
    # drop rows that contain node ['INNL']
    df.to_parquet(f'data/pearl_{vs_to}.parquet')

df


Processing overlay CCY
Processing overlay OVL
Processing overlay CCY
Processing overlay OVL
Processing overlay CCY
Processing overlay OVL


,date,component_id,portfolio_return,benchmark_return,portfolio_weight,benchmark_weight
0,2020-07-01,TOTAL/CA,-0.00026,-0.00000,-0.00464,0.00000
1,2020-07-02,TOTAL/CA,0.00009,-0.00000,-0.00336,0.00000
2,2020-07-03,TOTAL/CA,-0.00012,-0.00000,-0.00447,0.00000
3,2020-07-06,TOTAL/CA,0.00012,-0.00000,-0.00423,0.00000
4,2020-07-07,TOTAL/CA,-0.00007,0.00000,-0.00134,0.00000
...,...,...,...,...,...,...
27379,2025-06-24,TOTAL,0.00760,0.01013,1.00000,1.00000
27380,2025-06-25,TOTAL,-0.00197,-0.00237,1.00000,1.00000
27381,2025-06-26,TOTAL,0.00578,0.00691,1.00000,1.00000
27382,2025-06-27,TOTAL,0.00717,0.00857,1.00000,1.00000


In [ ]:
# CHILD	PARENT
# TOTAL	 
# EQLIKE	TOTAL
# EQ	EQLIKE
# EQSE	EQ
# EQSELC	EQSE
# EQSESC	EQSE
# EQDM	EQ
# EQDMLC	EQDM
# EQDMSC	EQDM
# EQEM	EQ
# HY	EQLIKE
# RE	EQLIKE
# RELI	RE
# RENL	RE
# IN	EQLIKE
# INLI	IN
# INNL	IN
# PE	EQLIKE
# HF	EQLIKE
# AS	EQLIKE
# OVL	EQLIKE
# IG	TOTAL
# CA	TOTAL
# CCY	TOTAL

<bound method NDFrame.head of      CHILD  PARENT                         PATH  IS_LEAF
0    TOTAL                                TOTAL    False
1   EQLIKE   TOTAL                 TOTAL/EQLIKE    False
2       EQ  EQLIKE              TOTAL/EQLIKE/EQ    False
3     EQSE      EQ         TOTAL/EQLIKE/EQ/EQSE    False
4   EQSELC    EQSE  TOTAL/EQLIKE/EQ/EQSE/EQSELC     True
5   EQSESC    EQSE  TOTAL/EQLIKE/EQ/EQSE/EQSESC     True
6     EQDM      EQ         TOTAL/EQLIKE/EQ/EQDM    False
7   EQDMLC    EQDM  TOTAL/EQLIKE/EQ/EQDM/EQDMLC     True
8   EQDMSC    EQDM  TOTAL/EQLIKE/EQ/EQDM/EQDMSC     True
9     EQEM      EQ         TOTAL/EQLIKE/EQ/EQEM     True
10      HY  EQLIKE              TOTAL/EQLIKE/HY     True
11      RE  EQLIKE              TOTAL/EQLIKE/RE    False
12    RELI      RE         TOTAL/EQLIKE/RE/RELI     True
13    RENL      RE         TOTAL/EQLIKE/RE/RENL     True
14      IN  EQLIKE              TOTAL/EQLIKE/IN    False
15    INLI      IN         TOTAL/EQLIKE/IN/INLI     True
1

In [ ]:
paths = {'leaf': ['TOTAL/EQLIKE/EQ/EQSE/EQSELC', 'TOTAL/EQLIKE/EQ/EQSE/EQSESC', 'TOTAL/EQLIKE/EQ/EQDM/EQDMLC', 'TOTAL/EQLIKE/EQ/EQDM/EQDMSC'],
         'node': ['TOTAL', 'TOTAL/EQLIKE', 'TOTAL/EQLIKE/EQ', 'TOTAL/EQLIKE/EQ/EQDM', 'TOTAL/EQLIKE/EQ/EQSE']}

rows = []

weight_type = 'relative'
for path_type, paths in paths.items():
    for path in paths:
        component_id = path.split('/')[-1]
        # print portfolio and benchmark weights for each component
        #print(f"Processing {component_id} with weight {get_latest_metric(df, component_id, 'portfolio_weight')}")
        rows.append({
            'id': component_id,
            'name': component_id,
            'type': path_type,
            'portfolio_return': get_latest_metric(df, component_id, 'portfolio_return'),
            'portfolio_weight': get_latest_metric(df, component_id, 'portfolio_weight'),
            'portfolio_weight_type': 'absolute' if component_id in overlays else weight_type,
            'benchmark_return': get_latest_metric(df, component_id, 'benchmark_return'),
            'benchmark_weight': get_latest_metric(df, component_id, 'benchmark_weight'),
            'benchmark_weight_type': 'absolute' if component_id in overlays else weight_type,
            'is_overlay': True if component_id in overlays else False,
            'meta': {'portfolio_return': get_historical_metric(df, component_id, 'portfolio_return'),
                    'benchmark_return': get_historical_metric(df, component_id, 'benchmark_return')},
            'path': path
        })

from spark.portfolio.graph import PortfolioBuilder

builder = PortfolioBuilder(normalize=True, enforce_sum_to_one=True, allow_shorts=True)

graph = (
    builder
    .from_paths(rows)
    .add_path(path='TOTAL/CCY', portfolio_weight=0.0, benchmark_weight=0.0, is_overlay=True, data={'portfolio_return': get_historical_metric(df, 'CCY', 'portfolio_return'),
                                                                                                      'benchmark_return': get_historical_metric(df, 'CCY', 'benchmark_return')})
    .build()
)

graph.print_tree()
from spark.portfolio.risk_analyzer import PortfolioRiskAnalyzer
analyzer = PortfolioRiskAnalyzer(graph)


Portfolio Tree Structure (Root: Portfolio Root)

└── 📁 Portfolio Root (portfolio) [P: 1.0000 | B: 1.0000 | A: +0.0000]
    └── 📁 TOTAL (TOTAL) [P: 1.0000 | B: 1.0000 | A: +0.0000]
        ├── 📁 EQLIKE (TOTAL/EQLIKE) [P: 0.8750 | B: 0.8712 | A: +0.0038]
        │   └── 📁 EQ (TOTAL/EQLIKE/EQ) [P: 1.0000 | B: 1.0000 | A: +0.0000]
        │       ├── 📁 EQSE (TOTAL/EQLIKE/EQ/EQSE) [P: 0.3124 | B: 0.3131 | A: -0.0007]
        │       │   ├── 🔸 EQSELC (TOTAL/EQLIKE/EQ/EQSE/EQSELC) [P: 0.7156 | B: 0.7235 | A: -0.0079]
        │       │   └── 🔸 EQSESC (TOTAL/EQLIKE/EQ/EQSE/EQSESC) [P: 0.2844 | B: 0.2765 | A: +0.0079]
        │       └── 📁 EQDM (TOTAL/EQLIKE/EQ/EQDM) [P: 0.6876 | B: 0.6869 | A: +0.0007]
        │           ├── 🔸 EQDMLC (TOTAL/EQLIKE/EQ/EQDM/EQDMLC) [P: 0.9826 | B: 0.9815 | A: +0.0011]
        │           └── 🔸 EQDMSC (TOTAL/EQLIKE/EQ/EQDM/EQDMSC) [P: 0.0174 | B: 0.0185 | A: -0.0011]
        └── 🔸 CCY (TOTAL/CCY) [P: 0.0000 | B: 0.0000 | A: +0.0000]

Legend:
  📁 = Portfolio Node 

In [ ]:
df.loc['TOTAL']

# Create random returns of len(df) and 3 columns
random_returns = pd.DataFrame(np.random.normal(0.01, 0.02, size=(len(df), 3)), index=df.index, columns=['Factor1', 'Factor2', 'Factor3'])
random_returns

Factor1  Factor2  Factor3
node  date                                 
AS    2020-07-01  0.02247  0.03085  0.01472
      2020-07-02  0.01168  0.01091 -0.01750
      2020-07-03  0.01795 -0.00552 -0.01006
      2020-07-06  0.03153 -0.01199 -0.01503
      2020-07-07  0.00518  0.03405  0.01270
...                   ...      ...      ...
TOTAL 2025-06-24 -0.00466 -0.01788  0.00398
      2025-06-25  0.02836  0.00630 -0.01257
      2025-06-26  0.06419  0.01821 -0.00233
      2025-06-27 -0.00585  0.00032  0.01815
      2025-06-30  0.00956 -0.00352 -0.02151

[31296 rows x 3 columns]

In [ ]:

root_component_id = 'TOTAL'
v = analyzer.decompose_factor_risk(root_component_id=root_component_id, factor_returns=factor_returns)
context = v.metric_store.get_all_metrics(root_component_id)['hierarchical_model_context'].value()
annualized_volatility = graph.excess_returns(root_component_id=root_component_id).value().std() * np.sqrt(252)
estimated_volatility = context.active_decomposer.risk_decomposition_summary()['portfolio_volatility']
print(f"Annualized Volatility: {annualized_volatility}, Estimated Volatility: {estimated_volatility}")
print(f"Portfolio Total Risk {context.portfolio_decomposer.portfolio_volatility}, Benchmark Total Risk: {context.benchmark_decomposer.portfolio_volatility}")
print(f"Portfolio Total Risk {graph.portfolio_returns(root_component_id=root_component_id).value().std() * np.sqrt(252)}, Benchmark Total Risk: {graph.benchmark_returns(root_component_id=root_component_id).value().std() * np.sqrt(252)}")


Annualized Volatility: 0.0124223530605765, Estimated Volatility: 0.007997890134428951
Portfolio Total Risk 0.1407133221697362, Benchmark Total Risk: 0.14002734179934748
Portfolio Total Risk 0.062225443571310865, Benchmark Total Risk: 0.06519059331667756


In [ ]:
for l, w, bw, aw in zip(context.descendant_leaves, context.active_decomposer.portfolio_weights, context.active_decomposer.benchmark_weights, context.active_decomposer.active_weights):
    print(f"Leaf: {l}, Weight: {w}, Benchmark Weight: {bw}, Active Weight: {aw}")



Leaf: TOTAL/EQLIKE/EQ/EQSE/EQSELC, Weight: 0.22354289691580898, Benchmark Weight: 0.22655393067212815, Active Weight: -0.0030110337563191703
Leaf: TOTAL/EQLIKE/EQ/EQSE/EQSESC, Weight: 0.08883657660124027, Benchmark Weight: 0.08656412582784355, Active Weight: 0.002272450773396717
Leaf: TOTAL/EQLIKE/EQ/EQDM/EQDMLC, Weight: 0.6756680368323202, Benchmark Weight: 0.6741723401995097, Active Weight: 0.0014956966328105725
Leaf: TOTAL/EQLIKE/EQ/EQDM/EQDMSC, Weight: 0.011952489650630487, Benchmark Weight: 0.012709603300518669, Active Weight: -0.0007571136498881817
Leaf: TOTAL/CCY, Weight: 0.0, Benchmark Weight: 0.0, Active Weight: 0.0
